In [1]:
from plant import Plant, RandomPlantData
from typing import List, Dict, Tuple
import numpy as np
from tqdm import tqdm

In [2]:
plant_data = RandomPlantData.generate_random_data()
file_path = 'trial.json'
RandomPlantData.generate_random_data_save(file_path)


plant = Plant.from_dictionary(plant_data)

In [3]:
for _ in tqdm(range(10**2)):
    a = RandomPlantData.generate_random_data()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 455.29it/s]


In [4]:
intervals_per_day = 24

def check_planification(plant, planification):
    '''
    check minimun stocks, minimun production times,
    possible transitions ...
    '''
    pass

def is_possible(plant, time, unit, grade, actual_grade, actual_grade_start):
    if t <= 10 * intervals_per_day and unit in plant.grades_after_10_days:
        return False

def order_simple_benefits(plant, order):
    '''
    returns the benefits regardless of transitions
    '''
    (grade, tons, price, priority) = order
    prod_flow = plant.prod_flow[grade, :]
    man_cost = plant.man_cost[grade, :]
    n_units = plant.n_units
    benefits = np.zeros(n_units)
    times = np.zeros(n_units)
    for unit in range(n_units):
        time = tons / prod_flow[unit]
        cost = man_cost[unit] * time
        benefits[unit] = price - cost
        times[unit] = time
    return benefits, times
     
order = plant.orders['firm']
order

{'22a3dafb-d35b-4413-ad0e-24ddff417b70': (18,
  1975.4033149573002,
  366.35112451263956,
  0.11971908461690606),
 'fbb42d44-95f8-4d23-b39f-9e67174c8d61': (7,
  1037.9759445157551,
  622.6408863809918,
  0.9162097568893723),
 '905365cc-2905-4da1-9c30-e21aeb899438': (14,
  1230.170622241318,
  86.87436820401696,
  0.726776442376906),
 '659c0521-44c6-433d-ba3c-9d39ded5be75': (5,
  437.21499337889173,
  724.4919537629281,
  0.535811831505534),
 'fc3163fd-96c4-4d4d-a8e6-9b6c52911b06': (13,
  726.0437332684141,
  346.97783558013975,
  0.5711146018291793),
 '356ba137-331d-4458-bdfb-3f733209c554': (18,
  528.3936205701362,
  654.9882872016683,
  0.6628419070131102),
 '377ab572-7515-4851-8945-a95e22023ca5': (16,
  319.9882368925684,
  648.4358654496159,
  0.21155114522811647),
 'fba49e3c-3b65-451f-8d80-cff434eb8d7a': (6,
  1006.0895731869439,
  368.0267518088358,
  0.9562364193340842),
 '6912cfb1-8d9d-4024-99a1-c50ad5d4196a': (3,
  322.02349886963975,
  946.5378959017755,
  0.38361069285973215

In [5]:
# %timeit {order_id: order_simple_benefits(plant, order) for order_id, order in plant.orders.items()}

In [6]:
'''(grade, tons, price, priority) = order
prod_flow = plant.prod_flow[grade, :]
man_cost = plant.man_cost[grade, :]
n_units = plant.n_units
benefits = np.zeros(n_units)
times = np.zeros(n_units)
for unit in range(n_units):
    time = tons / prod_flow[unit]
    cost = man_cost[unit] * time
    print(time, cost)
    benefits[unit] = price - cost
    times[unit] = time'''

'(grade, tons, price, priority) = order\nprod_flow = plant.prod_flow[grade, :]\nman_cost = plant.man_cost[grade, :]\nn_units = plant.n_units\nbenefits = np.zeros(n_units)\ntimes = np.zeros(n_units)\nfor unit in range(n_units):\n    time = tons / prod_flow[unit]\n    cost = man_cost[unit] * time\n    print(time, cost)\n    benefits[unit] = price - cost\n    times[unit] = time'

In [7]:
class Planification:
    def __init__(
            self,
            plant: "Plant",
            complete: bool = False,
            horizon: int = 30 * 24,
            orders_plan: Dict[int,
                              List[Tuple[int, int, int, float, int]]] = {},
            grades_plan: Dict[int, List[Tuple[int, int, int]]] = {},
            # TODO: Maintenance stops
    ):
        self.plant = plant
        self.complete = complete
        self.horizon = horizon
        
        if orders_plan:
            self.orders_plan = orders_plan
        else:
            self.orders_plan = {i: [] for i in range(self.plant.n_units)}
        
        if grades_plan:
            self.grades_plan = grades_plan
        else:
            self.grades_plan = {i: [] for i in range(self.plant.n_units)}
        
        self.orders = plant.orders['firm']
            
    def check_feasibility(self):
        '''
        check minimun stocks, minimun production times,
        possible transitions ...
        '''
        raise NotImplementedError('Method not implemented!')
        
    def calculate_benefits(self):
        raise NotImplementedError('Method not implemented!')
        


def calculate_minimum_time_to_transition(time, stocks):
    pass

In [8]:
def is_possible_transition(plant, grade, time, unit, actual_grade):
    if time <= 10 * intervals_per_day and unit in plant.grades_after_10_days:
        return False
    elif grade in plant.not_allowed_transitions.get(actual_grade, []):
        return False
    elif grade in plant.only_predecessor and plant.only_predecessor[grade] != actual_grade:
        return False
    elif unit == plant.unique_unit and grade in plant.unique_grades:
        return False
    else:
        return True


def calculate_possible_transitions(plant, grades, time, unit, actual_grade):
    possible_transitions = []
    for grade in grades:
        if is_possible_transition(plant, grade, time, unit, actual_grade):
            possible_transitions.append(grade)
    return possible_transitions


def group_orders_by_grade(orders):
    grade2orders = {}
    for order_id, order in orders.items():
        (grade, tons, price, priority) = order
        if grade in grade2orders:
            grade2orders[grade].add(order_id)
        else:
            grade2orders[grade] = set([order_id])
    return grade2orders

def calculate_min_stock_time_cost(plant, stocks, unit, grade):
    prod_flow = plant.prod_flow[grade, unit]
    man_cost = plant.man_cost[grade, unit]
    s_min = plant.s_min[grade]
    tons_left = max(0, s_min - stocks[grade])
    time = tons_left / prod_flow
    cost = man_cost * time
    return time, cost


# %timeit group_orders_by_grade(plant.orders)

In [9]:
print(plant_data.keys())

print(plant_data['t_min'], '____', plant_data['s_min'])

dict_keys(['n_grades', 'n_units', 'prod_flow', 'man_cost', 't_min', 's_min', 'only_consecutive', 'only_predecessor', 'not_allowed_transitions', 'orders', 'grades_after_10_days', 'unique_unit', 'unique_grades'])
[8.46379537437641, 4.13601229871427, 1.5154170892651762, 10.646787105714036, 3.12165550087192, 9.865652690953741, 1.2889123356363474, 5.696809756512626, 7.458020000020884, 8.018561740550116, 7.891451926910156, 9.358988366343175, 9.72826163016654, 1.4414184216541077, 1.9004129407797747, 2.64010479494146, 6.89776367229882, 4.507077761725098, 7.727603717223473, 3.7293108972237925] ____ [76.28741304006527, 28.77823053780956, 31.8635717858023, 26.50575586644717, 9.325402402829805, 40.53981617899847, 49.38525628482337, 62.12132767149718, 57.068913938307496, 30.125797724413033, 43.44462459040136, 24.78883398258383, 58.27830473975749, 77.63871978246218, 58.07709827825521, 13.110849237895495, 43.257028332730584, 64.18394029120148, 42.465676766471965, 56.54751468032172]


In [10]:
stocks = np.zeros(plant.n_grades)

# initial:
unit = 1
init = True
if init == True:
    possible_transitions = range(plant.n_grades)

time_to_change = 0
for grade in possible_transitions:
    s_time, s_cost = calculate_min_stock_time_cost(plant, stocks, unit, grade)
    time_to_change = plant.t_min[grade] #+ s_time
    print(grade, time_to_change, s_time, s_cost)
    #grade_orders = grade2orders[grade]

0 8.46379537437641 0.9125487318939757 15.156556463864826
1 4.13601229871427 0.26775627018984666 4.264219031474506
2 1.5154170892651762 0.30553390603888103 3.89381719811154
3 10.646787105714036 0.5233700239433472 7.865083434994461
4 3.12165550087192 0.057259994157139414 1.1829155307002657
5 9.865652690953741 0.17522046038472802 4.694839158807381
6 1.2889123356363474 0.23253557603256972 2.4238770810538375
7 5.696809756512626 0.3287093015229798 3.7583463398540182
8 7.458020000020884 1.2101220243283615 11.730076936474559
9 8.018561740550116 0.25030203868620454 2.350678301018951
10 7.891451926910156 0.42668086896582724 3.2624680441611633
11 9.358988366343175 0.1261126362761241 1.1512663902426337
12 9.72826163016654 0.7951142890072979 6.2711576053066915
13 1.4414184216541077 1.3398850114713683 35.02051467200635
14 1.9004129407797747 0.4162904189696147 6.964031389534219
15 2.64010479494146 0.07450774262970516 1.2442112956662739
16 6.89776367229882 0.1744292762261362 4.244994752090967
17 4.507

In [11]:
last_time = 30 * 24
def calculate_best_initial(interval=10, max_trans=3):
    pass


In [12]:

stocks = np.zeros(plant.n_grades)

for time in range(24 * 4):
    

IndentationError: expected an indented block (2923437359.py, line 4)

In [ ]:
t_0, 

In [ ]:
%timeit calculate_possible_transitions(plant, range(200), 24 * 12, 1, 3) # 6.67 µs

In [ ]:
{order_id: order_simple_benefits(plant, order)
 for order_id, order in plant.orders.items()}

In [ ]:
grade = 0
plant.prod_flow[grade, :]

In [ ]:
plant.not_allowed_transitions